In [ ]:
import torchaudio as ta
from chatterbox_infer.mtl_tts import ChatterboxMultilingualTTS
import time

model = ChatterboxMultilingualTTS.from_pretrained(device="cuda")

In [ ]:
from IPython.display import Audio

text = "hm.."
out = 'hmhm.wav'
wav, tokens = model.generate(text, audio_prompt_path='./output.wav', language_id='en')
print(wav.shape[-1]/24000)
ta.save(out, wav, model.sr)
display(Audio(out))

In [ ]:
from IPython.display import Audio
import torch

display(Audio(wav, rate=24000))
bi=0
all_audios = []
for i in [20, 40, 60, 80, 100, 120, 140, 160]:
    cwav, _ = model.s3gen.inference(
        speech_tokens=tokens[bi:i],
        ref_dict=model.conds.gen,
    )
    all_audios.append(cwav.detach().cpu())
    # display(Audio(cwav.detach().cpu(), rate=24000))
    bi = i

In [ ]:
summed_wav = torch.concat(all_audios, dim=1)
display(Audio(summed_wav, rate=24000))

In [ ]:
wav.shape

In [ ]:
import torchaudio as ta
from chatterbox.tts import ChatterboxTTS
import time
from IPython.display import Audio

model = ChatterboxTTS.from_pretrained(device="cuda")

In [ ]:
text = "Oh, I really love it.. y'know, it's amazing."

last_length = 0
st = time.time()
wavs = []
async for evt in model.generate_stream(text, audio_prompt_path='test-1.wav'):
    if evt["type"] == "chunk":
        wav = evt["audio"]
        wavs.append(wav[:, max(0, last_length-1000):])
        print(f"[{(wav.shape[-1]-last_length)/24000}s] - {time.time() - st}")
        last_length = wav.shape[-1]
        st = time.time()
    elif evt["type"] == "eos":
        print("✅ 스트리밍 끝!")

In [ ]:
import torch, time

SR = 24000
OVERLAP = int(0.05 * SR)  # 50ms

text = "Oh, I really love it.. y'know, it's amazing."

last_length = 0          # 모델이 지금까지 만든 전체 wav 길이
last_tail = None         # 직전 출력 청크의 꼬리(OVERLAP 샘플)
st = time.time()

wavs = []                # overlap-add로 섞은 최종 출력 청크들

async for evt in tts_model.generate_stream(text, audio_prompt_path='test-1.wav'):
    if evt["type"] == "chunk":
        wav = evt["audio"]                    # shape: (ch, T_total_so_far)
        device = wav.device
        dtype = wav.dtype

        # 이번에 "새로 추가된" 구간만 잘라오기
        new_total = wav.shape[-1]
        delta = new_total - last_length
        if delta <= 0:
            continue  # 새로 생긴 게 없으면 스킵

        new_part = wav[:, last_length:new_total]  # (ch, delta)

        if last_tail is None:
            # 첫 청크면 바로 내보냄
            out_chunk = new_part
        else:
            # 겹치는 길이 L (= min(OVERLAP, 새로 생긴 길이, last_tail 길이))
            L = min(OVERLAP, new_part.shape[-1], last_tail.shape[-1])
            if L > 0:
                # last_tail의 마지막 L 샘플 ↔ new_part의 앞 L 샘플을 교차페이드
                fade_in  = torch.linspace(0, 1, L, device=device, dtype=dtype)
                fade_out = 1.0 - fade_in

                mixed = last_tail[:, -L:] * fade_out + new_part[:, :L] * fade_in
                tail  = new_part[:, L:]  # 비겹침 뒷부분

                out_chunk = torch.cat([mixed, tail], dim=-1)
            else:
                # 겹칠 게 없으면 그냥 이어붙임
                out_chunk = new_part

        # 다음 교차페이드를 위해 꼬리 갱신
        # (모델 기준 전체 wav의 최신 꼬리를 쓰는 게 안전)
        new_tail_start = max(0, new_total - OVERLAP)
        last_tail = wav[:, new_tail_start:new_total].detach()

        # 사용자 출력/저장을 위해 overlap-add 결과만 모음
        wavs.append(out_chunk)

        print(f"[{out_chunk.shape[-1]/SR:.3f}s] - {time.time() - st:.3f}")
        last_length = new_total
        st = time.time()

    elif evt["type"] == "eos":
        print("✅ 스트리밍 끝!")

In [ ]:
import torch

display(Audio(wav.cpu().numpy(), rate=24000))   # 예: 오디오 출력 함수
display(Audio(torch.concat(wavs, dim=-1).cpu().numpy(), rate=24000))   # 예: 오디오 출력 함수

In [ ]:
st = time.time()
model.t3.speech_pos_emb.get_fixed_embedding(213)
time.time() - st

import torch

st = time.time()
emb = torch.nn.Embedding(113, 512)
emb(torch.tensor(12))
time.time() - st

In [ ]:
st = time.time()
torch.tensor(132).to('cuda')
time.time() - st

In [ ]:
logits = torch.randn((132001))
st = time.time()
probs = torch.softmax(logits, dim=-1)
next_token = torch.multinomial(probs, num_samples=1)  # shape: (B, 1)
print(time.time() - st)

st = time.time()
dist = torch.distributions.Categorical(logits=logits)  # GPU에서 동작
next_token = dist.sample()  # (B, 1) 형태 맞추기
print(time.time() - st)


In [ ]:
import librosa
import time

audio, sr = librosa.load('test-1.wav')

text = "Oh"
st = time.time()
wav, tokens = model.generate(text, audio_prompt_path=audio)
print(wav.shape, tokens.shape, time.time() - st)
# ta.save("test-1.wav", wav, model.sr)
display(Audio('test-1.wav'))

In [ ]:
import torchaudio as ta
from chatterbox_infer.tts import ChatterboxTTS
import time

model = ChatterboxTTS.from_pretrained(device="cuda")

In [ ]:
from IPython.display import Audio
import librosa
import torch

audio, _ = librosa.load('/workspace/chatterbox/sesame.wav', sr=16000, mono=True)

text = "I want to go home now please let me go."
wav, tokens = model.generate(text, audio_prompt_path=audio)
print(wav.shape[-1]/24000)

out = 'just.wav'
ta.save(out, wav, model.sr)
display(Audio(out))

In [ ]:
display(Audio(audio, rate=16000))

In [ ]:

# 지금 계속 한글로 말하는걸 영어로 번역하고 있어.
# <previous utterances>는 현재 문장 이전에 이야기하던 문장이야. 번역을 위한 맥락 파악에 사용할 수 있어.
# <speaking english>은 번역해야하는 현재 발화야.

# 말을 하는걸 script로 만든 input이기 때문에, 발음 문제로 인해서 텍스트가 잘못 들어왔을 수 있어. 그걸 감안해서 번역해줘.

# 출력 english를 일반 글 문장보다는 실제로 사람이 말하는 것 같은 구어체로 적어줘. 예를 들어, 같은 단어를 두번 쓰거나 뭐 ...을 쓰거나 느낌표 이런 것들 있잖아?
# Translate into casual spoken English. 근데 너무 심하게 하진 말고, 없는 말을 만들거나 들어온 input을 왜곡하면 안돼.
# Do not start with word like Oh, So, Uhm, Huh, etc.

# 혹시 한글 문장이 아직 종결되지 않았는데 중간에 들어온 것 같으면, 출력 영어도 종결되지 않고 뒤에 이어질 수 있는 문장으로 뱉어줘.
# 뒤에 어떤 말이 들어올지 모르니까 만약 문장이 종결되지 않았다고 판단되면, 뒤에 어떤 문장이 들어오던 현재 출력하는 문장을 이어서 완성할 수 있게 해야해.
# 그래서 꼭 지금 한글 input을 전부 번역할 필요는 없어.

# 현재까지 번역된 문장이 있으면, 그 문장을 이어서 번역해줘. 먼저 들어온 번역 문장이 잘못 되었을 수도 있어. 그래도 알아서 잘 구어체로.
# 추가 번역되어야하는 문장만 출력해줘.

# 그리고 만약 뒤에 어떤 다른 문장이 들어오던 현재의 speaking korean을 전부 <english>에 포함시켜서 번역이 완료되었다면, 마지막에 <END>를 붙여줘.
# 뒤에 추가로 번역이 필요하면 ...을 마지막에 붙여줘.

# -- INPUT --
# <previous utterances>{hist}
# <speaking korean> : {current_scripted_sentence}
# <english> : {current_translated}


In [ ]:
import os
from typing import Callable
from openai import OpenAI
import time

OPENAI_KEY = os.environ.get("OPENAI_KEY")

client = OpenAI(api_key=OPENAI_KEY)

def translate(prevScripts:str, current_scripted_sentence:str, current_translated:str, onToken, input_language:str = 'Korean', output_language:str = 'English'):
    hist = "\n".join([f" me:{x}," for x in prevScripts])
    
    response = client.chat.completions.create(
        model='gpt-4.1-mini',  # 최신 경량 모델
        messages=[
            {"role": "system", "content": f"You are a professional translator specializing in [{input_language}] → [{output_language}] translation."},
            {"role": "user", "content": f"""
You are translating {input_language} speech into {output_language}.

<previous utterances> are the sentences spoken before the current one. Use them for context.  
<speaking korean> is the current spoken input that needs to be translated.  
<{output_language}> is the translation generated so far.  

The input comes from speech-to-text, so there may be transcription errors due to pronunciation. Please take this into account when translating.  

Output the translation in casual spoken {output_language} — like how people actually talk, with natural pauses, repetitions, or fillers such as “...” or “!” — but don’t overdo it. Do not invent new words or distort the original meaning.  
Do not start with words like “Oh”, “So”, “Uhm”, or “Huh”.  

If the {input_language} input seems incomplete (cut off mid-sentence), output an unfinished {output_language} sentence too, so it can be naturally continued. You don’t need to force a full translation of every fragment if it isn’t complete yet.
한글 종결 어미의 특징 : ~~요. ~~니다. ~~어.

If there is already some translated {output_language}, continue from it. !Do not include the previous translation in the output never!
The earlier translation may not be perfect — refine it naturally into spoken {output_language}. Only output the additional translated part.  

If the current {input_language} input is fully translated and nothing else needs to be added, end with `<END>`.  
If more input is likely to follow, end with `...`.  

# Real-Time Translation Tips
1. **Avoid Premature Subject Translation**
- Korean often omits or ambiguates the subject.
- When the subject is unclear, try to infer it from prior context.
- If there’s even slight ambiguity, avoid explicitly translating the subject ("I", "we", "they", etc.) and use neutral or impersonal expressions instead.
- Example:
  - Korean: “마케팅비를 청구해야 한다”
  - Preferred: “Marketing costs must also be claimed.<END>”
  - Not: “I should claim the marketing costs.<END>”
- Example:
  - Korean: "내일 집에"
  - Preferred: "Tomorrow,"
  - Not: "Tomorrow at home, I'll "

2. Do not include the verb in the translation if no verb is spoken.
- Korean places verbs at the end. Don't translate prematurely if the action is unknown.
- Example: “운동장에가서 축구를…”
  - Preferred: "I'll go to the sports field and..." # Still don't know whether they will play soccer or watch. Just skip the sentence, because there will be more input to come next.
  - Not: "I'll go to the sports field and play soccer..." # This is not correct, because next input can be "축구를 봤어."

-- INPUT --  
<previous utterances> {hist}  
<speaking {input_language}> : {current_scripted_sentence}  
<{output_language}> : {current_translated}  
"""}
        ],
        temperature=0.1,
        user="k2e-translator-v1-hojinkhj6051230808",
        prompt_cache_key="k2e-translator-v1-hojinkhj6051230808",
        stream=True,
        stream_options={"include_usage": True},
    )

    sent = ''
    first = 0
    st = time.time()

    pt = 0
    pt_cached = 0
    ct = 0

    for chunk in response:
        if chunk.usage and chunk.usage is not None:
            u = chunk.usage;
            pt += u.prompt_tokens
            pt_cached += u.prompt_tokens_details.cached_tokens
            ct += u.completion_tokens
        else:
            if chunk.choices[0].delta.content != '' and chunk.choices[0].delta.content is not None:
                onToken(chunk.choices[0].delta.content)
                sent += chunk.choices[0].delta.content

    return {
        "text": sent,
        "prompt_tokens": pt,
        "prompt_tokens_cached": pt_cached,
        "completion_tokens": ct
    }

In [ ]:
# from llm.openai import translate
import time

examples = [
    ['"저는 다음주 쯤에 샌프란시스코로.', '가려다가 그냥 안가려구요.'],
    ['이번 주에 회의가 있을 수도 있어.', '밥 뭐먹지?'],
    ['오늘 아침에 밥을', '먹진 않고 간단하게 음료수만 마셨어.'],
    ['제가 항상 듣던 말은.', '너는 왜그렇게 말이 많냐는 거였어요.'],
]

for ex in examples:
    output1 = translate([], ex[0], "", lambda x: None)
    output2 = translate([], f"{ex[0]} {ex[1]}", output1['text'], lambda x: None)

    print(output1)
    print(output2, "\n")